In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

## Shopify DS Intern Question 1: Sneaker Sales

In [2]:
# Import the data

data = pd.read_csv('./shopify_data.csv')

In [3]:
data.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [4]:
data[['order_amount']].describe()

,order_amount
count,5000.000000
mean,3145.128000
std,41282.539349
min,90.000000
25%,163.000000
50%,284.000000
75%,390.000000
max,704000.000000


### Analysis at a glance

From the description of the data, it's clear that there are outliers in the order_amount and total_items. There seems to be (at least) 1 purchase of 2000 sneakers, totaling $704000.

A reasonable metric to use might be the median order_amount ($284). The median naturally adjusts for outliers, but has the drawback of limiting our analysis. The other solution I've come up with is to get rid of the outliers then find the mean. This lets us make projections about change that we aren't able to do with median.

In [5]:
high_outliers = data[data['order_amount'] >= np.percentile(data['order_amount'], 99)]

print("High outliers: ", high_outliers.shape)

High outliers:  (63, 7)


In [6]:
print("Number of shops in the top 1% of distribution: ", len(high_outliers['shop_id'].unique()))
print("Shop IDs: ", high_outliers['shop_id'].unique())

Number of shops in the top 1% of distribution:  2
Shop IDs:  [42 78]


In [7]:
# Shop_id : 42 has made 17 sales of 704000. 
# Now lets look at shop_id : 78

high_outliers[high_outliers['shop_id'] == 42]

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00
1104,1105,42,607,704000,2000,credit_card,2017-03-24 4:00:00
1362,1363,42,607,704000,2000,credit_card,2017-03-15 4:00:00
1436,1437,42,607,704000,2000,credit_card,2017-03-11 4:00:00
1562,1563,42,607,704000,2000,credit_card,2017-03-19 4:00:00
1602,1603,42,607,704000,2000,credit_card,2017-03-17 4:00:00
2153,2154,42,607,704000,2000,credit_card,2017-03-12 4:00:00
2297,2298,42,607,704000,2000,credit_card,2017-03-07 4:00:00


After looking deeper into the top 1% of orders on Shopify, we can see that they are all made by two vendors (shop_id 78 and shop_id 42). These two are extreme outliers and should be excluded from the AOV.

#### Solution 1: Find the median!

The advantage of using the median over the mean is that the median adjusts for outliers. Rather than taking the average sale price per order, we are taking the sale price at the 50th percentile. 

In [8]:
print("Median order value: ", np.median(data['order_amount']))

Median order value:  284.0


While this metric is helpful because it adjusts for outliers, there's a downside. Using the median order value we are unable to make projections about how much the average order value changes overall. What it is actually telling us is how much the middle value has changed. (An explaination can be found here: https://www.nctm.org/tmf/library/drmath/view/75818.html)

#### Solution 2: Exclude the outliers!

Since it appears that the only outliers are in shop 42 and 78, I will simply elimate those orders from the dataset and reassess.

In [9]:
# Exclude shop 42 and 78

data_excluding_outliers = data[(data['shop_id'] != 42) & (data['shop_id'] != 78)]

In [10]:
# We lost 97 values, about 2% of the data.

data_excluding_outliers.shape

(4903, 7)

In [11]:
# Check the describe() function to see the new mean

data_excluding_outliers.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,4903.000000,4903.000000,4903.000000,4903.000000,4903.000000
mean,2499.584540,49.900877,849.858862,300.155823,1.995717
std,1444.221163,29.154367,86.887947,155.941112,0.982602
min,1.000000,1.000000,700.000000,90.000000,1.000000
25%,1246.500000,24.000000,776.000000,163.000000,1.000000
50%,2499.000000,50.000000,850.000000,284.000000,2.000000
75%,3750.500000,74.000000,925.000000,386.500000,3.000000
max,5000.000000,100.000000,999.000000,1086.000000,8.000000


With a mean of \\$300.15, only $15 higher than the median, we are pretty safe in assuming there aren't extreme outliers skewing our data.